In [1]:
!pip install stable-baselines3==2.1.0 gymnasium[atari] ale-py imageio matplotlib

INFO: pip is looking at multiple versions of gymnasium[atari] to determine which version is compatible with other requirements. This could take a while.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 178.7/178.7 kB 13.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 953.9/953.9 kB 51.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 66.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 71.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 62.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 46.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 12.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/

In [2]:
!pip install gymnasium[accept-rom-license]

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 434.7/434.7 kB 25.6 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for AutoROM.accept-rom-license: filename=autorom_accept_rom_license-0.6.1-py3-none-any.whl size=446709 sha256=f732f8d6fa3bd45bf919788094db5ce038e893b6827fe219d6ae13eb3dfee8d4
  Stored in directory: /root/.cache/pip/wheels/bc/fc/c6/8aa657c0d2089982f2dabd110efc68c61eb49831fdb7397351
Successfully built AutoROM.accept-rom-license


In [4]:
import gymnasium as gym
from stable_baselines3 import DQN
from stable_baselines3.common.env_util import make_atari_env
from stable_baselines3.common.vec_env import VecFrameStack
from stable_baselines3.common.callbacks import CheckpointCallback
from stable_baselines3.common.logger import configure

env_id = "BreakoutNoFrameskip-v4"

# Create Atari environment with 4 parallel instances
env = make_atari_env(env_id, n_envs=4, seed=42)
env = VecFrameStack(env, n_stack=4)

# Use MLP policy
policy = "MlpPolicy"

model = DQN(
    policy,
    env,
    learning_rate=1e-3,  # Faster learning
    gamma=0.99,
    batch_size=64,  # Process more samples per step
    buffer_size=10_000,  # Smaller buffer
    exploration_initial_eps=1.0,
    exploration_final_eps=0.05,  # Reduce exploration decay speed
    exploration_fraction=0.2,
    target_update_interval=500,  # Update more frequently
    train_freq=1,  # Train more often
    gradient_steps=2,  # More gradient updates per step
    verbose=1,
    tensorboard_log="./dqn_tensorboard/"
)

# Configure logging
logger = configure("./dqn_logs_mlp/", ["stdout", "csv", "tensorboard"])
model.set_logger(logger)

# Checkpoint callback
checkpoint_callback = CheckpointCallback(save_freq=5000, save_path="./checkpoints_mlp/")

# Train for 500,000 steps
model.learn(
    total_timesteps=500_000,
    callback=checkpoint_callback,
    tb_log_name="dqn_breakout_mlp_fast"
)

# Save the trained model
model.save("dqn_breakout_mlp_fast")

env.close()


Streaming output truncated to the last 5000 lines.
| train/              |          |
|    learning_rate    | 0.001    |
|    loss             | 0.000351 |
|    n_updates        | 202462   |
----------------------------------
----------------------------------
| rollout/            |          |
|    ep_len_mean      | 666      |
|    ep_rew_mean      | 1.2      |
|    exploration_rate | 0.05     |
| time/               |          |
|    episodes         | 13568    |
|    fps              | 263      |
|    time_elapsed     | 1724     |
|    total_timesteps  | 455132   |
| train/              |          |
|    learning_rate    | 0.001    |
|    loss             | 0.00138  |
|    n_updates        | 202564   |
----------------------------------
----------------------------------
| rollout/            |          |
|    ep_len_mean      | 670      |
|    ep_rew_mean      | 1.23     |
|    exploration_rate | 0.05     |
| time/               |          |
|    episodes         | 13572    |
|   